# Task 1 – K-NN classification

## Task 1.1

In [10]:
import numpy as np
import scipy.io as sio                       # for loading matlab files
from collections import Counter

In [2]:
path = "../data.mat"
data = sio.loadmat(path) # load data

(Xtrn255, Ctrn) = data['dataset']['train'][0][0][0][0] # maps to training set & labels
#                                                       (strange to have to go so deep)
(Xtst255, Ctst) = data['dataset']['test'][0][0][0][0]  # maps to test set & labels

# normalise values as directed:
Xtrn = np.array(np.divide(Xtrn255, 255.0), dtype=np.float_)
Ctrn = np.array(Ctrn, dtype=np.int_)

Xtst = np.array(np.divide(Xtst255, 255.0), dtype=np.float_)
Ctst = np.array(Ctst, dtype=np.int_)

# not using types
# Xtrn = np.array(np.divide(Xtrn255, 255.0))
# Ctrn = np.array(Ctrn)

# Xtst = np.array(np.divide(Xtst255, 255.0))
# Ctst = np.array(Ctst)

In [3]:
def MySqDist(Xtrn, Xtst):
# optimise by expressing as matrix operarions
#     (Xtrn-Xtst)^2 = Xtrn^2 + Xtst^2 - 2(Xtrn * Xtst)
    
#     Xtrn^2 = np.sum(Xtrn ** 2, axis=1)[:, np.newaxis] # lengths of all training vectors
    
#     Xtst^2 = np.sum(Xtst ** 2, axis=1)                # lengths of all test vectors
    
#     Xtrn * Xtst = np.dot(Xtrn, Xtst.T)                # matrix product of sums of products
    
    return (np.sum(Xtrn ** 2, axis=1)[:, np.newaxis]
          + np.sum(Xtst ** 2, axis=1)
          - 2 * np.dot(Xtrn, Xtst.T))

In [4]:
a = np.array([[1,2,3],
              [4,5,6]])
b = np.array([[1,2,3],
              [4,5,6],
              [7,8,9]])
print(a.shape, b.shape)
# print(a)
# print(b)
%time print MySqDist(b,a)
from sklearn.metrics.pairwise import euclidean_distances
%time euclidean_distances(b, a)

((2, 3), (3, 3))
[[  0  27]
 [ 27   0]
 [108  27]]
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 378 µs
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 545 µs


array([[ 0.        ,  5.19615242],
       [ 5.19615242,  0.        ],
       [10.39230485,  5.19615242]])

In [5]:
%time cached_cpreds = MySqDist(Xtrn, Xtst)

CPU times: user 35.3 s, sys: 5.14 s, total: 40.4 s
Wall time: 11.9 s


In [157]:
def my_knn_classify_old(Xtrn, Ctrn, Xtst, Ks):
    # Input:
    #   Xtrn : M-by-D ndarray of training data (dtype=np.float_)
    #   Ctrn : M-by-1 ndarray of labels for Xtrn (dtype=np.int_)
    #   Xtst : N-by-D ndarray of test data (dtype=np.float_)
    #   Ks   : List of the numbers of nearest neighbours in Xtrn
    # Output:
    #  Cpreds : N-by-L ndarray of predicted labels for Xtst (dtype=np.int_)
    
#     Cpreds_shape = (Ks.shape, Xtst.shape[0])
    
    Cpreds = np.empty((0, Xtst.shape[0]))
#     print Cpreds.shape
#     print Cpreds
    
#     (x, y) = cached_cpreds.shape
    
#     cl_type = np.dtype([('sqdist', np.float_, (x,)), ('class', np.int_, 1)])           # specify end type
    
#     dists = np.empty((x,), dtype=cl_type)
    
    d = cached_cpreds[:2000]
    c = Ctrn[:2000]
    
#     print d[:10]
    cl_sort = np.argsort(d, axis=0)     # get minimum value indices as columns
    
    for (i, k) in enumerate(Ks):
        inds = cl_sort[:k]                                        # grab k nearest from columns
#         print inds
#         print inds
#         print Ctrn[inds][:,]
    
#         print Counter(np.array([1,2,1]))
    
#         print Ctrn[inds][0:2,0:]
        preds = np.apply_along_axis(lambda x: max(Counter(x)), 1, Ctrn[inds].T[0]).T.reshape(1, Xtst.shape[0])    # count occurrences 
        
#         print preds.shape
#         print preds
    
        Cpreds = np.concatenate((Cpreds[:i], preds, Cpreds[i:]))
        
#         for row in Ctrn[inds].T[0].T:
#             unique, counts = np.unique(row, return_counts=True)
#             Cpreds.append(max(zip(unique, counts)))

#         unique, counts = np.unique(Ctrn[inds].T[0].T, return_counts=True)
        
#         print zip(unique, counts)
    
#         print Counter(map(tuple, Ctrn[inds].T[0].T))
        
#         print Counter([1,2,1,3])
#         print Counter(Ctrn[inds].T.tolist())
#         unique, counts = np.unique(Ctrn[inds][:,], return_counts=True)  # count occurrences
#         cs = dict(zip(unique, counts))                              #
        
#         print cs
#         print(sorted(cs, key=cs.get))
#         print(max(cs, key=cs.get))
        
#         for 
        
#         Cpreds.append(max(cs, key=cs.get))
#         add result to output var
#         k_row.append(sorted(Counter(sorted_c[:k]).items())[0][0])
#             k_row.append(Counter(sorted_c[:k]))

#         Cpreds.append(k_row)
#     Cpreds.append(dists)
        
    return Cpreds


In [200]:
def my_knn_classify(Xtrn, Ctrn, Xtst, Ks, use_cache=False):
    # Input:
    #   Xtrn : M-by-D ndarray of training data (dtype=np.float_)
    #   Ctrn : M-by-1 ndarray of labels for Xtrn (dtype=np.int_)
    #   Xtst : N-by-D ndarray of test data (dtype=np.float_)
    #   Ks   : List of the numbers of nearest neighbours in Xtrn
    # Output:
    #  Cpreds : N-by-L ndarray of predicted labels for Xtst (dtype=np.int_)
    
    Cpreds = np.empty((0, Xtst.shape[0]))               # set shape of prediction matrix
    
    if use_cache:                       # use caching system while developing to reduce computation time
        try:
            d = cached_cpreds[:10000]           # set distance matrix to cached
        except Exception:
            d = MySqDist(Xtrn, Xtst)    # cache does not exist, calculate distance matrix
    else:
        d = MySqDist(Xtrn, Xtst)        # no-cache mode, skip and calculate distance matrix
    
    cl_sort = np.argsort(d, axis=0, kind='quicksort')     # get minimum value indices as columns
    
    for (i, k) in enumerate(Ks):
        inds = cl_sort[:k]      # grab k nearest from columns
        
        preds = np.apply_along_axis(    # apply counting function along each row
            lambda x: max(Counter(x)),  # get most likely classification from possibilities
            1, 
            Ctrn[inds].T[0]).T.reshape(1, Xtst.shape[0])
#              |         |                   |
#             \/        \/                  \/
#      grab letter    transpose      transpose the result,
#      number from    and shed       reshape to allow for
#    closest vector    a layer     concatenation with output

#         preds = np.apply_along_axis(lambda x: max(Counter(x)), 1,Ctrn[inds].T[0]).T.reshape(1, Xtst.shape[0])
# #              |         |                   |
# #             \/        \/                  \/
# #      grab letter    transpose      transpose the result,
# #      number from    and shed       reshape to allow for
# #    closest vector    a layer     concatenation with output
        
        Cpreds = np.concatenate((Cpreds[:i], preds, Cpreds[i:]))
        
    return Cpreds


In [201]:
%time knn_c = my_knn_classify(Xtrn, Ctrn, Xtst, np.array([2]), use_cache=True) # [0][1][0]).reshape(20,5)
print knn_c.shape
print knn_c


CPU times: user 6.42 s, sys: 438 ms, total: 6.86 s
Wall time: 6.86 s
(1, 7800)
[[14. 24. 19. ... 10.  5. 26.]]


In [ ]:
# %time knn_c = my_knn_classify(Xtrn, Ctrn, Xtst, [1,3,5,10,20]) # [0][1][0]).reshape(20,5)
# knn_c

In [ ]:
# from sklearn.metrics.pairwise import euclidean_distances
# %time np.square(euclidean_distances(Xtrn, Xtst))

In [ ]:
# "Minimal code for rendering a numpy array as an image in a Jupyter
#  notebook in memory. Borrowed from the Deep Dream notebook."

# https://gist.github.com/kylemcdonald/2f1b9a255993bf9b2629

# used to display characters as images

import PIL.Image
from cStringIO import StringIO
import IPython.display
import numpy as np
def showarray(a, fmt='png'):
    a = np.uint8(a)
    f = StringIO()
    PIL.Image.fromarray(a).save(f, fmt)
    IPython.display.display(IPython.display.Image(data=f.getvalue()))

In [ ]:
import string

for (i, sample) in enumerate(Xtst255[:100]):
    showarray(np.reshape(sample, (28, 28)).T)
    c = int(Ctrn['class'][i])
    print c
    print string.ascii_uppercase
    print(string.ascii_uppercase[c-1])
# for pred in 
    
# np.reshape(Xtrn[0], (28, 28))

In [ ]:
a = np.array([[1,2,3],
              [4,5,6]])
b = np.array([[1,2,3],
              [4,5,6],
              [7,8,9]])
print(a.shape, b.shape)
# print(a)
# print(b)
%time print MySqDist(b,a)
from sklearn.metrics.pairwise import euclidean_distances
%time euclidean_distances(b, a)

In [ ]:
from sklearn.neighbors import NearestNeighbors

nbrs = NearestNeighbors(n_neighbors=2, algorithm='auto').fit(Xtrn[:100])
distances, indices = nbrs.kneighbors(Xtrn)

print distances

In [ ]:
zzzz = np.array([[1,2,3,4,5],
                 [1,1,1,1,2],
                 [2,2,4,1,3]])
# zzzz = zzzz + zzzz.T


print zzzz

# zzz = Counter(np.argsort(zzzz, axis=0)[:3])

print np.argsort(zzzz, axis=0)[:3]

for row in np.argsort(zzzz, axis=0)[:3].T:
#     print row
    print Ctrn[0], Ctrn[40], Ctrn[30], 
    print Ctrn[[0,40,30]]
    print ""
    unique, counts = np.unique(Ctrn[[0,40,30]], return_counts=True) # Ctrn[:5000][row]
    print unique
    print counts
    print "---"
#     print unique, counts
    